# 🦌 Deer Detector v2.0 — YOLO Training Notebook

**Project:** Deer Deterrent System  
**Model:** YOLO26s (real YOLO26) or YOLOv8s (fallback)  
**Dataset:** v2.0 with CLAHE preprocessing + negatives  
**Hardware:** Colab T4 GPU  

## Steps:
1. Setup environment
2. Upload dataset
3. Phase 1: Frozen backbone training (20 epochs)
4. Phase 2: Full fine-tune (130 epochs)
5. Evaluate on test set
6. Export for deployment (ONNX + OpenVINO)
7. Download trained model

## 1. Setup Environment

In [ ]:
# Install dependencies
!pip install -q ultralytics>=8.4.0 opencv-python-headless

# Verify
import ultralytics
print(f"Ultralytics: {ultralytics.__version__}")

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

## 2. Upload Dataset

Upload your `dataset_v2.0.tar.gz` exported by `scripts/export_dataset_v2.py`.

**Option A:** Upload directly to Colab (temporary — lost when session ends)  
**Option B:** Upload to Google Drive first (persistent)

In [ ]:
# Option A: Direct upload
from google.colab import files
import os

# Create dataset directory
DATASET_DIR = "/content/dataset"
os.makedirs(DATASET_DIR, exist_ok=True)

print("Upload dataset_v2.0.tar.gz:")
uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    if filename.endswith('.tar.gz'):
        !tar xzf "{filename}" -C {DATASET_DIR}
        print(f"Extracted {filename} to {DATASET_DIR}")
    elif filename.endswith('.zip'):
        !unzip -q "{filename}" -d {DATASET_DIR}
        print(f"Extracted {filename} to {DATASET_DIR}")

# Verify
!ls -la {DATASET_DIR}/
!cat {DATASET_DIR}/data.yaml

In [ ]:
# Option B: From Google Drive (uncomment if using Drive)
# from google.colab import drive
# drive.mount('/content/drive')
# 
# DATASET_DIR = "/content/dataset"
# os.makedirs(DATASET_DIR, exist_ok=True)
# !tar xzf /content/drive/MyDrive/deer-deterrent/dataset_v2.0.tar.gz -C {DATASET_DIR}
# !cat {DATASET_DIR}/data.yaml

In [ ]:
# Verify dataset structure
import os
from pathlib import Path

DATASET_DIR = "/content/dataset"
ds = Path(DATASET_DIR)

for split in ['train', 'val', 'test']:
    imgs = list((ds / 'images' / split).glob('*.jpg'))
    lbls = list((ds / 'labels' / split).glob('*.txt'))
    pos = sum(1 for l in lbls if l.stat().st_size > 0)
    neg = len(lbls) - pos
    print(f"  {split:5s}: {len(imgs)} images, {len(lbls)} labels (positive: {pos}, negative: {neg})")

print(f"\n  Total: {sum(len(list((ds/'images'/s).glob('*.jpg'))) for s in ['train','val','test'])} images")

## 3. Phase 1: Frozen Backbone Training (20 epochs)

Freeze the COCO-pretrained backbone and only train the detection head.
This lets the head adapt to deer-specific features without destroying
the useful low-level feature extractors.

In [ ]:
from ultralytics import YOLO
import json
from datetime import datetime

DATASET_DIR = "/content/dataset"
DATA_YAML = f"{DATASET_DIR}/data.yaml"
OUTPUT_DIR = "/content/runs"
RUN_NAME = f"deer_v2_{datetime.now().strftime('%m%d_%H%M')}"

# Try YOLO26s first, fallback to YOLOv8s
try:
    model = YOLO('yolo26s.pt')
    ARCH = 'YOLO26s'
    print(f"✅ Loaded YOLO26s (real YOLO26)")
except Exception as e:
    print(f"⚠️ YOLO26s not available ({e}), using YOLOv8s")
    model = YOLO('yolov8s.pt')
    ARCH = 'YOLOv8s'
    print(f"✅ Loaded YOLOv8s (fallback)")

print(f"\nArchitecture: {ARCH}")
print(f"Run name: {RUN_NAME}")

In [ ]:
# Phase 1: Frozen backbone
results_p1 = model.train(
    data=DATA_YAML,
    epochs=20,
    imgsz=640,
    batch=16,
    device='cuda',
    project=OUTPUT_DIR,
    name=f"{RUN_NAME}_p1",
    
    # Freeze backbone
    freeze=10,
    
    # Optimizer
    optimizer='AdamW',
    lr0=0.01,
    lrf=0.01,
    weight_decay=0.0005,
    warmup_epochs=3,
    
    # Augmentation
    hsv_h=0.02,
    hsv_s=0.7,
    hsv_v=0.5,
    translate=0.15,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    
    patience=0,  # No early stopping
    verbose=True,
    plots=True,
)

print("\n✅ Phase 1 complete!")

## 4. Phase 2: Full Fine-tune (130 epochs)

Unfreeze all layers and train the full network with a lower learning rate.
Early stopping patience=30 prevents overfitting.

In [ ]:
from pathlib import Path

# Load best model from Phase 1
p1_best = Path(OUTPUT_DIR) / f"{RUN_NAME}_p1" / "weights" / "best.pt"
if not p1_best.exists():
    p1_best = Path(OUTPUT_DIR) / f"{RUN_NAME}_p1" / "weights" / "last.pt"

print(f"Loading Phase 1 best: {p1_best}")
model2 = YOLO(str(p1_best))

# Phase 2: Full fine-tune
results_p2 = model2.train(
    data=DATA_YAML,
    epochs=130,
    imgsz=640,
    batch=16,
    device='cuda',
    project=OUTPUT_DIR,
    name=f"{RUN_NAME}_p2",
    
    # No freeze — full network
    freeze=0,
    
    # Lower LR for fine-tuning
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    weight_decay=0.0005,
    warmup_epochs=0,
    
    # Augmentation (slightly stronger)
    hsv_h=0.02,
    hsv_s=0.7,
    hsv_v=0.5,
    translate=0.15,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.15,
    copy_paste=0.1,
    
    patience=30,  # Early stopping
    verbose=True,
    plots=True,
)

print("\n✅ Phase 2 complete!")

## 5. Evaluate on Test Set

In [ ]:
# Load final best model
final_best = Path(OUTPUT_DIR) / f"{RUN_NAME}_p2" / "weights" / "best.pt"
if not final_best.exists():
    final_best = Path(OUTPUT_DIR) / f"{RUN_NAME}_p1" / "weights" / "best.pt"

print(f"Evaluating: {final_best}")
eval_model = YOLO(str(final_best))

# Test set evaluation
metrics = eval_model.val(
    data=DATA_YAML,
    split='test',
    imgsz=640,
    batch=16,
    device='cuda',
    verbose=True,
)

print(f"\n{'='*50}")
print(f"Test Results:")
print(f"  mAP50:     {metrics.box.map50:.4f}")
print(f"  mAP50-95:  {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall:    {metrics.box.mr:.4f}")
print(f"{'='*50}")

In [ ]:
# Visualize some test predictions
import glob
from IPython.display import Image, display

test_images = sorted(glob.glob(f"{DATASET_DIR}/images/test/pos_*.jpg"))[:6]

if test_images:
    results = eval_model.predict(
        source=test_images,
        conf=0.25,
        save=True,
        project=OUTPUT_DIR,
        name=f"{RUN_NAME}_test_viz",
    )
    
    # Show results
    viz_dir = Path(OUTPUT_DIR) / f"{RUN_NAME}_test_viz"
    for img_path in sorted(viz_dir.glob('*.jpg'))[:6]:
        display(Image(filename=str(img_path), width=640))
        print(f"  {img_path.name}")
else:
    print("No positive test images found for visualization")

## 6. Export for Deployment

Export to ONNX for the Dell server deployment.  
OpenVINO conversion should be done on the server itself for best compatibility.

In [ ]:
# Export to ONNX
deploy_model = YOLO(str(final_best))

# ONNX export
onnx_path = deploy_model.export(format='onnx', imgsz=640, simplify=True)
print(f"\n✅ ONNX exported: {onnx_path}")

# Show model size
import os
pt_size = os.path.getsize(str(final_best)) / 1e6
onnx_size = os.path.getsize(str(onnx_path)) / 1e6 if onnx_path else 0
print(f"\nModel sizes:")
print(f"  PyTorch (.pt):  {pt_size:.1f} MB")
print(f"  ONNX (.onnx):   {onnx_size:.1f} MB")

In [ ]:
# Try OpenVINO export (may fail on Colab — that's OK, do it on the Dell server)
try:
    !pip install -q openvino openvino-dev
    ov_path = deploy_model.export(format='openvino', imgsz=640, half=True)
    print(f"✅ OpenVINO FP16 exported: {ov_path}")
except Exception as e:
    print(f"⚠️ OpenVINO export failed on Colab (expected): {e}")
    print("Export on Dell server instead:")
    print("  yolo export model=best.pt format=openvino half=True imgsz=640")

## 7. Download Trained Model

Download `best.pt` (and optionally ONNX) to your local machine,
then copy to Dell server:

```bash
scp best.pt rndpig@192.168.7.215:/home/rndpig/deer-deterrent/dell-deployment/models/best.pt
ssh rndpig@192.168.7.215 "cd /home/rndpig/deer-deterrent && docker compose restart ml-detector"
```

In [ ]:
from google.colab import files
import shutil

# Copy best model to a convenient location
download_dir = Path('/content/download')
download_dir.mkdir(exist_ok=True)

# Copy best.pt
shutil.copy2(str(final_best), download_dir / 'best.pt')
print(f"Copied best.pt ({final_best.stat().st_size / 1e6:.1f} MB)")

# Copy ONNX if available
onnx_file = final_best.with_suffix('.onnx')
if onnx_file.exists():
    shutil.copy2(str(onnx_file), download_dir / 'best.onnx')
    print(f"Copied best.onnx ({onnx_file.stat().st_size / 1e6:.1f} MB)")

# Save training summary
summary = {
    'architecture': ARCH,
    'dataset_version': '2.0',
    'test_map50': float(metrics.box.map50),
    'test_map50_95': float(metrics.box.map),
    'test_precision': float(metrics.box.mp),
    'test_recall': float(metrics.box.mr),
    'model_size_mb': final_best.stat().st_size / 1e6,
    'training_timestamp': datetime.now().isoformat(),
}
with open(download_dir / 'training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\nTraining Summary:")
for k, v in summary.items():
    print(f"  {k}: {v}")

# Download
print("\n📥 Downloading files...")
files.download(str(download_dir / 'best.pt'))
files.download(str(download_dir / 'training_summary.json'))

## 8. Deployment Instructions

After downloading `best.pt`:

```bash
# 1. Copy model to Dell server
scp best.pt rndpig@192.168.7.215:/home/rndpig/deer-deterrent/dell-deployment/models/best.pt

# 2. SSH into server
ssh rndpig@192.168.7.215
cd /home/rndpig/deer-deterrent

# 3. Update ml-detector Dockerfile to use ultralytics>=8.4.0
# (required for YOLO26s inference)

# 4. Add CLAHE preprocessing to ml-detector inference
# (must match training preprocessing)

# 5. Rebuild and restart
docker compose build ml-detector
docker compose up -d --force-recreate ml-detector

# 6. Test with a known deer image
curl -X POST http://localhost:8001/detect -F 'file=@test_deer.jpg'
```

### Important: CLAHE must be applied during inference too!
The model was trained on CLAHE-enhanced images. Add this to the ml-detector's `/detect` endpoint:

```python
import cv2
import numpy as np

def enhance_ir_image(image: np.ndarray) -> np.ndarray:
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced_l = clahe.apply(l)
    return cv2.cvtColor(cv2.merge([enhanced_l, a, b]), cv2.COLOR_LAB2BGR)
```